# Web Scrapping de dados de apartamentos para alugar na cidade de Florianópolis SC no site Viva Real

*Web Scrapping of data about apartaments for rent in Florianópolis (Santa Catarina Departament, Brazil) whitin the website called "Viva Real"*

by Haisa Vargas Echeli.

Importando as bibliotecas necessárias
*(Importing the needed packages)*

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
import pandas as pd
from tqdm import tqdm
from datetime import datetime

Instalando o WebDriver Manager para o  navegador Google Chrome *(Installing the WebDriver Manager for Google Chrome browser)*

In [2]:
#Isso também abre o navegador (this also opens the browser)!
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\lisaf\.wdm\drivers\chromedriver\win32\90.0.4430.24]


Definindo variáveis *(Creating the variables)*

In [3]:
url_dict = {"Florianópolis": "https://www.vivareal.com.br/aluguel/santa-catarina/florianopolis/apartamento_residencial/"}

errors = []
results = []

Web Scrapping dos dados do site *(Web Scrapping the site data)*

In [4]:
for city, url in url_dict.items():
    current_url = url
    driver.get(url)
    sleep(2)
    actions = ActionChains(driver)
#Primeiro, vamos garantir aceitar os cookies da página (First, let's make sure to accept the cookies on the page) 
    try:
        driver.find_element_by_class_name("cookie-notifier__cta").click()
    except:
            print("No cookies!")
    #Agora vamos percorrer diversas páginas de resultados da pesquisa no website 
    #(Now we are going to scroll through several search results pages on the website)
    for i in tqdm(range(144), desc=city):
    
    #Primeiramente eu rodei com 200 páginas, mas retornou erro pois só vai até a página 145 nesse caso, 
    #então para pegar toda a informação eu escolhi colocar exatamente 144 na opção range.
    #(First I've tried with 200 pages, but it return an error because it only goes to 145 pages in this case, 
    #so to get all the information, I've choosed to put it exactly 144 in the range option.)   
        
        sleep(5)
        main_div = driver.find_element_by_class_name("results-main__panel")
        properties = main_div.find_elements_by_class_name("js-property-card")
        paginator = driver.find_element_by_class_name("js-results-pagination")
        next_page = paginator.find_element_by_xpath("//a[@title='Próxima página']") #/html/body/main/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/a
        
        #Percorrer cada apartamento da página e guardar as informações desejadas
        #(Scroll through each apartment on the page and save the wanted information)
        for i,apartment in enumerate(properties):
            url = apartment.find_element_by_class_name("js-card-title").get_attribute("href")
            apto_id = url.split("id-")[-1][:-1]
            header = apartment.find_element_by_class_name("property-card__title").text
            address = apartment.find_element_by_class_name("property-card__address").text
            area = apartment.find_element_by_class_name("js-property-card-detail-area").text
            rooms = apartment.find_element_by_class_name("js-property-detail-rooms").text
            bathrooms = apartment.find_element_by_class_name("js-property-detail-bathroom").text
            garages = apartment.find_element_by_class_name("js-property-detail-garages").text
            #Tratar casos onde não há amenidades (Treat cases where there are no amenities):
            try:
                amenities = apartment.find_element_by_class_name("property-card__amenities").text
            except:
                amenities = None
                errors.append(url)
            price = apartment.find_element_by_class_name("js-property-card-prices").text
            #Tratar casos onde não há preço do condomínio (Treat cases where there is no condo fee):
            try:
                condo = apartment.find_element_by_class_name("js-condo-price").text
            except:
                condo = None
                errors.append(url)
            crawler = city
            crawled_at = datetime.now().strftime("%Y-%m-%d %H:%M")
            #Criar dicionário com as informações de cada apartamento 
            #(Create dictionary with the information of each apartment)
            results.append({"id": apto_id,
                            "url": url,
                            "header": header,
                            "address": address,
                            "area": area,
                            "rooms": rooms,
                            "bathrooms": bathrooms,
                            "garages": garages,
                            "amenities": amenities,
                            "price": price,
                            "condo": condo,
                            "crawler": crawler,
                            "crawled_at": crawled_at})
        #Clicar na próxima página pesquisada (Click on the next searched page):
        next_page.click()
#Criar um data frame, no formato csv, no Pandas com os dados coletados e fechar o navegador!  
#(Create a data frame, in csv format, in Pandas with the collected data and close the browser!)
pd.DataFrame(results).to_csv("full_results.csv", index=False)
driver.close()



Florianópolis: 100%|█████████████████████████████████████████████████████████████████| 144/144 [46:49<00:00, 19.51s/it]


In [5]:
df = pd.DataFrame(results)

In [6]:
df.shape

(5184, 13)

In [7]:
df.head()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
0,2520673504,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 75m²","Rua Coronel Pedro Demoro, 1507 - Balneario do ...",75,2 Quartos,2 Banheiros,1 Vaga,Elevador\nPiscina\nChurrasqueira\nSalão de fes...,R$ 2.800 /mês,None,Florianópolis,2021-05-24 01:25
1,2512632181,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 65m²","Rua Maestro Aldo Krieger, 54 - Córrego Grande,...",65,2 Quartos,2 Banheiros,1 Vaga,Elevador\nSalão de festas\nAr-condicionado\nCo...,R$ 1.700 /mês,R$ 740,Florianópolis,2021-05-24 01:25
2,2517219101,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto para Aluguel, 35m²","Rua Maria Eduarda, 33 - Pantanal, Florianópoli...",35,1 Quarto,1 Banheiro,-- Vaga,Mobiliado\nElevador\nAr-condicionado\nCondomín...,R$ 1.400 /mês,R$ 300,Florianópolis,2021-05-24 01:25
3,2518663689,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto para Aluguel, 70m²","Servidão Manoel Isidoro Augusto, 560 - Rio Tav...",70,1 Quarto,1 Banheiro,1 Vaga,Jardim\nSistema de alarme\nCozinha americana,R$ 1.900 /mês,None,Florianópolis,2021-05-24 01:25
4,2516104017,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 4 Quartos para Aluguel, 250m²","Avenida dos Búzios, 3147 - Jurerê Internaciona...",250,4 Quartos,3 Banheiros,2 Vagas,Mobiliado\nElevador\nChurrasqueira\nSalão de f...,R$ 5.200 /mês,R$ 1.200,Florianópolis,2021-05-24 01:25


In [8]:
df.tail()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
5179,2513969525,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com Quarto para Aluguel, 55m²","Rua Tenente Silveira, 324 - Centro, Florianópo...",55,1 Quarto,1 Banheiro,-- Vaga,Elevador,R$ 1.100 /mês,R$ 428,Florianópolis,2021-05-24 02:12
5180,2518970781,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 70m²","Trindade, Florianópolis - SC",70,2 Quartos,1 Banheiro,-- Vaga,Churrasqueira,R$ 1.500 /mês,R$ 370,Florianópolis,2021-05-24 02:12
5181,2512250283,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 91m²","Rua Almirante Lamego, 830 - Centro, Florianópo...",91,2 Quartos,1 Banheiro,1 Vaga,Elevador\nÁrea de serviço\nInterfone,R$ 1.650 /mês,R$ 660,Florianópolis,2021-05-24 02:12
5182,2519080590,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 75m²","Rodovia Tertuliano Brito Xavier, 688 - Canasvi...",75,2 Quartos,1 Banheiro,1 Vaga,Elevador\nPlayground\nAcademia\nQuadra poliesp...,R$ 1.700 /mês,R$ 318,Florianópolis,2021-05-24 02:12
5183,2514496364,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 50m²","Rua Trajano Margarida - Trindade, Florianópoli...",50,2 Quartos,1 Banheiro,1 Vaga,None,R$ 1.500 /mês,R$ 120,Florianópolis,2021-05-24 02:12


In [9]:
df.describe()

,id,url,header,address,area,rooms,bathrooms,garages,amenities,price,condo,crawler,crawled_at
count,5184,5184,5184,5184,5184,5184,5184,5184,3591,5184,3720,5184,5184
unique,5184,5184,477,1333,213,5,6,7,1049,522,751,1,48
top,2516334093,https://www.vivareal.com.br/imovel/apartamento...,"Apartamento com 2 Quartos para Aluguel, 70m²","Canasvieiras, Florianópolis - SC",60,2 Quartos,1 Banheiro,1 Vaga,Elevador,R$ 1.500 /mês,R$ 350,Florianópolis,2021-05-24 01:28
freq,1,1,212,206,267,2298,3045,3409,628,243,110,5184,144
